<a href="https://colab.research.google.com/github/amalianuruljannah/TugasSosmed1/blob/main/Tugas_Sosmed_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# --- Install library tambahan (jika perlu) ---
!pip install tqdm openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
from tqdm import tqdm
from google.colab import files
from datetime import datetime

# HEADERS untuk menyamar sebagai browser
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- Scraping detail artikel ---
def scrape_article_detail(url):
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        if response.status_code != 200:
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Isi artikel
        content_div = soup.find('div', class_='detail-text')
        if not content_div:
            return None
        paragraphs = content_div.find_all('p')
        full_content = ' '.join([p.get_text(strip=True) for p in paragraphs])

        # Published Date
        date_elem = soup.find('div', class_='date')
        if date_elem:
            published_date = date_elem.get_text(strip=True)
        else:
            # fallback ke meta tag
            meta_date = soup.find("meta", attrs={"property": "article:published_time"})
            if meta_date and meta_date.get("content"):
                raw_date = meta_date["content"]
                try:
                    dt = datetime.fromisoformat(raw_date.replace("Z", "+00:00"))
                    published_date = dt.strftime("%d %B %Y, %H:%M WIB")
                except:
                    published_date = raw_date
            else:
                published_date = "Tanggal tidak ditemukan"

        return {
            'published_date': published_date,
            'content': full_content
        }
    except Exception as e:
        print("Error detail:", e)
        return None

# --- Scraping daftar artikel ---
def scrape_cnn_indeks(total_pages=2):
    all_articles = []
    base_url = 'https://www.cnnindonesia.com/indeks'

    for page in range(1, total_pages + 1):
        index_url = f"{base_url}?page={page}" if page > 1 else base_url
        response = requests.get(index_url, headers=HEADERS, timeout=10)
        if response.status_code != 200:
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        articles_on_page = soup.find_all('article')

        for article_tag in tqdm(articles_on_page, desc=f"Scraping Halaman {page}"):
            link_tag = article_tag.find('a')
            if not link_tag or not link_tag.has_attr('href'):
                continue
            article_url = link_tag['href']

            title_tag = article_tag.find('h2')
            title = title_tag.get_text(strip=True) if title_tag else 'N/A'

            detail_data = scrape_article_detail(article_url)

            if detail_data:
                article_data = {
                    'title': title,
                    'published_date': detail_data['published_date'],
                    'article_url': article_url,
                    'content': detail_data['content'],
                    'source_page': page
                }
                all_articles.append(article_data)

            time.sleep(0.5)

    return all_articles

# --- Eksekusi ---
scraped_data = scrape_cnn_indeks(total_pages=2)

if scraped_data:
    df = pd.DataFrame(scraped_data)

    # Simpan CSV
    csv_filename = 'cnn_scraped.csv'
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    files.download(csv_filename)

    # Simpan JSON
    json_filename = 'cnn_scraped.json'
    with open(json_filename, "w", encoding="utf-8") as f:
        json.dump(scraped_data, f, ensure_ascii=False, indent=4)
    files.download(json_filename)

    # Simpan XLSX
    xlsx_filename = 'cnn_scraped.xlsx'
    df.to_excel(xlsx_filename, index=False)
    files.download(xlsx_filename)

    print("✅ Data berhasil diekspor & siap di-download!")
    display(df.head())
else:
    print("❌ Tidak ada data yang berhasil di-scrape.")


Scraping Halaman 1:   0%|          | 0/34 [00:00<?, ?it/s]

Error detail: Invalid URL '#': No scheme supplied. Perhaps you meant https://#?


Scraping Halaman 1:   3%|▎         | 1/34 [00:00<00:16,  2.00it/s]

Error detail: Invalid URL '#': No scheme supplied. Perhaps you meant https://#?


Scraping Halaman 1:   6%|▌         | 2/34 [00:01<00:16,  2.00it/s]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Data berhasil diekspor & siap di-download!


,title,published_date,article_url,content,source_page
0,Menhut Buat Tim Percepatan Penetapan Hutan Ada...,Tanggal tidak ditemukan,https://www.cnnindonesia.com/nasional/20250915...,Menteri Kehutanan (Menhut) RI Raja Juli Antoni...,1
1,Hasil Seleksi Awal Rekrutmen Damkar DKI Diumum...,Tanggal tidak ditemukan,https://www.cnnindonesia.com/nasional/20250915...,Pemerintah Provinsi (Pemprov)DKI Jakartaakan m...,1
2,KPU Beber Rahasiakan 16 Dokumen Capres-Cawapre...,Tanggal tidak ditemukan,https://www.cnnindonesia.com/nasional/20250915...,KetuaKPUMochammad Afifuddin mengungkap alasan ...,1
3,Polda Metro Jaya Buka Posko Pengaduan Orang Hi...,Tanggal tidak ditemukan,https://www.cnnindonesia.com/nasional/20250915...,Polda Metro Jaya membuka posko pengaduanorang ...,1
4,"Serabut Putih pada Lapisan Daging Jeruk, Amank...",Tanggal tidak ditemukan,https://www.cnnindonesia.com/gaya-hidup/202509...,"Saat digigit, daging buahjerukterasa menyegark...",1
